In [1]:
import cv2
import numpy as np

In [4]:
img1 = None
win_name = 'Camera Matching'
min_match = 10

In [5]:
# ORB 검출기 생성
detector = cv2.ORB_create(1000)

In [7]:
# Flann 추출기 생성
flann_index_lsh = 6
index_params = dict(algorithm = flann_index_lsh, table_number = 6,
                   key_size = 12, multi_probe_level = 1)
search_params = dict(checks = 32)
matcher = cv2.FlannBasedMatcher(index_params, search_params)

In [8]:
# 카메라 캡쳐 연결 및 프레임 크기 축소
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

False

In [ ]:
while cap.isOpened():
    ret, frame = cap.read()
    if img1 is None:  # 등록된 이미지 없음, 매칭 과정 없이 카메라 영상을 곧바로 출력
        res = frame
    else:             # 등록된 이미지가 있는 경우, 매칭 시작
        img2 = frame
        gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
        gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
        # 키 포인트와 디스크립터 추출
        kp1, desc1 = detector.detectAndCompute(gray1, None)
        kp2, desc2 = detector.detectAndCompute(gray2, None)
        # knnMatch, k=2
        matches = matcher.knnMatch(desc1, desc2, 2)
        # 이웃 거리의 75%로 좋은 매칭점 추출
        ratio = 0.75
        good_matches = [m[0] for m in matches 
                        if len(m) == 2 and m[0].distance < m[1].distance * ratio]
        print(f'good matches:{len(good_matches):d}/{len(matches):d}')
        # 모든 매칭점을 그리지 못하게 마스크를 0으로 채움
        matchesMask = np.zeros(len(good_matches)).tolist()
        # 좋은 매칭점이 최소 개수 이상인 경우
        if len(good_matches) > min_matches:
            # 좋은 매칭점으로 원본과 대상 영상의 좌표 구하기
            src_pts = np.float32([kp1[m.queryIdx].pt for m in good_matches])
            dst_pts = np.float32([kp2[m.trainIdx].pt for m in good_matches])
            # 원근 변환행렬 구하기
            mtrx, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
            accuracy = float(mask.sum()) / mask.size
            print(f'accuracy:{mask.sum():d}/{mask.size:d}({accuracy:.2f}%)')
            if mask.sum() > min_match:  # 정상치 매칭점이 최소 개수 이상인 경우
                # 이상점 매칭점만 그리게 마스크 설정
                matchesMask = mask.ravel().tolist()